Data Preparation

In [1]:
import pyprind
import pandas as pd
import os
import sys

basepath = 'aclImdb'

In [2]:
labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000, stream=sys.stdout)
df = pd.DataFrame()
data = []
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile: 
                txt = infile.read()
            #df = df.append([[txt, labels[l]]],ignore_index=True)
            data.append([txt, labels[l]])
            pbar.update()

df = pd.DataFrame(data)
#df.columns = ['review', 'sentiment']

In [3]:
df.columns = ['review', 'sentiment']

In [6]:
import numpy as np
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('movie_data.csv', index=False, encoding='utf-8')